In [3]:
!pip install -r requirement.txt

ERROR: Could not find a version that satisfies the requirement python-doten (from versions: none)
ERROR: No matching distribution found for python-doten


In [2]:
from dotenv import load_dotenv
load_dotenv()


True

In [4]:
import os
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')
# os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['GROQ_API_KEY'] = os.getenv('GROK_API_KEY')

In [10]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    model="qwen-qwq-32b")

print(llm)

client=<groq.resources.chat.completions.Completions object at 0x000001EFBC7CCB90> async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001EFBCE28650> model_name='qwen-qwq-32b' model_kwargs={} groq_api_key=SecretStr('**********')


In [11]:
print(llm.invoke("What is the capital of France?"))

content="\n<think>\nOkay, so I need to figure out what the capital of France is. Let's start by recalling what I know about France. France is a country in Western Europe, right? I remember learning about famous cities there like Paris, Lyon, Marseille, and maybe Bordeaux. But which one is the capital?\n\nHmm, I think Paris is the most well-known city in France. But wait, sometimes countries have capitals that aren't the most famous city. For example, I know that the capital of the United States is Washington D.C., not New York, which is bigger and more famous. But in France's case, I'm pretty sure Paris is the capital. Let me think of other examples. Like how London is the capital of the UK, even though cities like Birmingham or Manchester are big too. So maybe it's similar with Paris and France.\n\nI should also consider if there's any recent changes. I don't recall hearing about France moving their capital anywhere else. The Eiffel Tower, the Louvre, the French government buildings—t

In [17]:
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_messages([
    ("system","You are a expert AI engineer. Provide a detailed answer to the question."),
    ("user","{input}")])

In [18]:
chain = prompt | llm

In [19]:
chain

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a expert AI engineer. Provide a detailed answer to the question.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001EFBC7CCB90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001EFBCE28650>, model_name='qwen-qwq-32b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [20]:
print(chain.invoke({"input":"Can you explain the concept of a neural network?"}))

content='\n<think>\nOkay, so I need to explain what a neural network is. Hmm, where do I start? I remember that neural networks are inspired by the human brain\'s structure, right? They have layers of nodes or neurons, kind of like how neurons in our brain are connected. But I\'m a bit fuzzy on the specifics. Let me think step by step.\n\nFirst, maybe I should start with the basic components. There\'s an input layer where data comes in, then maybe some hidden layers where the processing happens, and an output layer that gives the result. Activation functions must be involved to introduce non-linearity. Oh, and each connection between neurons has weights, which are adjusted during training. But how exactly do they learn?\n\nWait, the process of training involves adjusting the weights to minimize error. That\'s probably through backpropagation. But what\'s backpropagation exactly? It\'s a method to calculate the gradient of the loss function with respect to the weights, right? And then u

In [9]:
from langchain_groq import ChatGroq
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain_core.messages import HumanMessage

# Setup your model
llm = ChatGroq(
    model="gemma2-9b-it"
)

# Output parser for comma-separated list
parser = CommaSeparatedListOutputParser()
format_instructions = parser.get_format_instructions()

# Prompt with explicit instruction
prompt = f"""
List five popular programming languages **only as a comma-separated list**.

{format_instructions}
"""

# Ask the model
response = llm.invoke([HumanMessage(content=prompt)])

# Parse the response
parsed = parser.parse(response.content)
print(parsed)


['Python', 'JavaScript', 'Java', 'C++', 'C# ']


### Assisgment:
Create a simple assistant that uses any LLM and should be pydantic, when we ask about any product it should give you two information product Name, product details tentative price in USD (integer). use chat Prompt Template.


In [17]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

# Step 1: Define the Pydantic model
class Product(BaseModel):
    name: str = Field(description="The name of the product")
    price: float = Field(description="The price of the product in USD")
    description: str = Field(description="A brief description of the product")

# Step 2: Initialize your LLM
llm = ChatGroq(model="gemma2-9b-it")

# Step 3: Setup output parser
parser = PydanticOutputParser(pydantic_object=Product)

# Step 4: Prompt Template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that provides product information."),
    ("user", "Provide details about the following product: {product}.\n{format_instructions}")
])

# Step 5: Chain prompt + model + parser
chain = prompt | llm | parser


product_query = "Samsung Galaxy S24"
result = chain.invoke({"product": product_query, "format_instructions": parser.get_format_instructions()})
print(result)

name='Samsung Galaxy S24' price=799.99 description='The Samsung Galaxy S24 is a flagship smartphone featuring a powerful processor, vibrant display, and advanced camera system.'


In [18]:
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"description": "The name of the product", "title": "Name", "type": "string"}, "price": {"description": "The price of the product in USD", "title": "Price", "type": "number"}, "description": {"description": "A brief description of the product", "title": "Description", "type": "string"}}, "required": ["name", "price", "description"]}\n```'

In [21]:
from langchain.output_parsers import YamlOutputParser
YamlOutputParser(pydantic_object=Product).get_format_instructions()

'The output should be formatted as a YAML instance that conforms to the given JSON schema below.\n\n# Examples\n## Schema\n```\n{"title": "Players", "description": "A list of players", "type": "array", "items": {"$ref": "#/definitions/Player"}, "definitions": {"Player": {"title": "Player", "type": "object", "properties": {"name": {"title": "Name", "description": "Player name", "type": "string"}, "avg": {"title": "Avg", "description": "Batting average", "type": "number"}}, "required": ["name", "avg"]}}}\n```\n## Well formatted instance\n```\n- name: John Doe\n  avg: 0.3\n- name: Jane Maxfield\n  avg: 1.4\n```\n\n## Schema\n```\n{"properties": {"habit": { "description": "A common daily habit", "type": "string" }, "sustainable_alternative": { "description": "An environmentally friendly alternative to the habit", "type": "string"}}, "required": ["habit", "sustainable_alternative"]}\n```\n## Well formatted instance\n```\nhabit: Using disposable water bottles for daily hydration.\nsustainabl